In [1]:
import numpy as np
np.random.seed(42)

In [2]:
import tensorflow as tf
import pdb
import fxns
reload(fxns)

<module 'fxns' from '/Users/yakir/Dropbox/Harvard G3-G4/cs282r/motifnet/code/fxns.pyc'>

In [5]:
# constants
data_path = '../data/examples'
out_path = '../output/examples'
train_out_path = '%s/texttrain' % out_path
valid_out_path = '%s/textvalidate' % out_path
test_out_path = '%s/texttest' % out_path
test_chrs = [1,2]
valid_chrs = [3,4]
blocksize = 10
num_blocks = 4
create_data = True
text = False
binary = True
batch_size = 40
capacity = 2000
min_after_dequeue = 1000
learning_rate = 0.01

In [6]:
# create data
if create_data:
    fxns.create_shards(data_path,
            train_out_path, valid_out_path, test_out_path,
            test_chrs, valid_chrs,
            blocksize=blocksize,
            text=text, binary=binary, num_blocks=num_blocks)

1
2
3
4
5
1
2
3
4
5
1
2
3
4
5


In [16]:
# get training data
tf.reset_default_graph()
(seq, label), info = fxns.get_seq_and_label(train_out_path)
seq_batch, label_batch = tf.train.shuffle_batch([seq, label], batch_size=batch_size,
    capacity=capacity,
    min_after_dequeue=min_after_dequeue)

# get validation data
(seqv, labelv), infov = fxns.get_seq_and_label(valid_out_path)
seqv_batch, labelv_batch = tf.train.shuffle_batch([seqv, labelv], batch_size=batch_size,
    capacity=capacity,
    min_after_dequeue=min_after_dequeue)

In [17]:
# print data
with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    _seq_batch, _label_batch = sess.run([seq_batch, label_batch])
#     print _seq_batch, _label_batch
    print _seq_batch.shape, _label_batch.shape
    coord.request_stop()
    coord.join(threads)

(40, 4000) (40, 24)


In [18]:
# define model
loss = fxns.get_logreg_model(seq_batch, label_batch, info['seq_len'], info['label_len'])
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)

In [28]:
# train

with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    sess.run(tf.global_variables_initializer())
    
    for i in range(100):
        _, _loss, a, b = sess.run([optimizer, loss, seq_batch, label_batch])
        av, bv = sess.run([seqv_batch,labelv_batch])
        _lossv = sess.run(loss,
            feed_dict={
                seq_batch:av,
                label_batch:bv
            })
        print np.where(b)[1], np.where(bv)[1]
        print i, _loss, _lossv
    
    coord.request_stop()
    coord.join(threads)

[15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15] [15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15]
0 0.703413 0.711689
[15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15] [15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15]
1 0.709294 0.676093
[15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15] [15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15]
2 0.680789 0.656195
[15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15] [15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15

 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15] [15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15]
32 0.277558 0.28551
[15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15] [15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15]
33 0.273039 0.277667
[15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15] [15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15]
34 0.27034 0.275323
[15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15] [15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15]
35 0.259314 0.266683
[15 15 15 

 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15] [15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15]
73 0.133595 0.149659
[15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15] [15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15]
74 0.135767 0.147447
[15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15] [15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15]
75 0.130587 0.14764
[15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15] [15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15]
76 0.133503 0.141395
[15 15 15